In [7]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install gradio
!pip install pymupdf

  Using cached llama_index_embeddings_huggingface-0.5.3-py3-none-any.whl.metadata (767 bytes)
Using cached llama_index_embeddings_huggingface-0.5.3-py3-none-any.whl (9.0 kB)


In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


#2.Loading the model and defining it's parameters

In [9]:
Settings.embed_model = HuggingFaceEmbedding(model_name= "BAAI/bge-small-en-v1.5")
Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [10]:

pdf_loader = PyMuPDFReader()
pages = pdf_loader.load_data("/content/BotGenesis/MATRIX_CRITICAL_REVIEW_cinema&philosophy_SE22UCSE106.pdf")


In [11]:
print(len(pages))

13


In [12]:
index = VectorStoreIndex.from_documents(pages)

In [13]:
top_k = 13

retriever = VectorIndexRetriever(
    index = index,
    similarity_top_k = top_k,
)


In [14]:
query_engine = RetrieverQueryEngine(
    retriever = retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)]
)

In [15]:
query = "Who embodies the quote IGNORANCE IS BLISS"
response = query_engine.query(query)

In [16]:
context = "Context: \n"
for i in range(top_k):
  context = context + response.source_nodes[i].text + "\n\n"
print(context)

Context: 
MATRIX (1999)
12
The Oracle said it better than anyone in the movie. She did not tell Neo that he was the one like
everyone around him did. She told him that he would have to choose between him or Morpheus,
as a test to see if he would blindly believe her or challenge his fate by defying her.
She always knew he was the one, as she told Morpheus that he was the one who would find the
One, and that Trinity would be in love with the man who was the One. But until Neo knew it
himself, he was not the One. Once he knew, he was the One.
As she herself said,

I believe Neo to be the one BECAUSE he believed in himself against all odds, and knew the
reality of the world, and not the latter happening due to him being the One.
This is a message to the viewer. There is a “One” among all of us, waiting to be discovered. You
could have the power to change the history of the world, but it wouldn’t matter if you believe the
world around you when it tells you that you can’t be whatever you wan

In [17]:
print(len(response.source_nodes))  # Check how many chunks were retrieved


13


In [18]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_pr

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [19]:
# prompt (no context)
intstructions_string = f"""EnigmaGPT, functioning as a chatbot to answer questions on your personalized text material, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–EnigmaGPT'. \
EnigmaGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "Who embodies the quote IGNORANCE IS BLISS?" #Replace with the same question as used in query above

prompt = prompt_template(comment)
print(prompt)

[INST] EnigmaGPT, functioning as a chatbot to answer questions on your personalized text material, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–EnigmaGPT'. EnigmaGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Who embodies the quote IGNORANCE IS BLISS? 
[/INST]


In [20]:
# Ensure pad token is set
tokenizer.pad_token = tokenizer.eos_token  # Set PAD token
model.config.pad_token_id = tokenizer.pad_token_id

# Set model to evaluation mode
model.eval()  # ✅ This ensures the model behaves correctly for inference

# Tokenize input with padding and truncation
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512  # Set an appropriate length
)

# Generate output with proper attention mask
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"),
    attention_mask=inputs["attention_mask"].to("cuda"),
    max_new_tokens=280
)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] EnigmaGPT, functioning as a chatbot to answer questions on your personalized text material, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–EnigmaGPT'. EnigmaGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
Who embodies the quote IGNORANCE IS BLISS? 
[/INST]
I'd say the character Ignorance from the fairy tale "The Emperor's New Clothes" embodies the quote "Ignorance is bliss." –EnigmaGPT</s>


In [21]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]EnigmaGPT, functioning as a chatbot to answer questions on your personalized text material, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–EnigmaGPT'. \
EnigmaGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""

prompt = prompt_template_w_context(context, comment)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> [INST]EnigmaGPT, functioning as a chatbot to answer questions on your personalized text material, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–EnigmaGPT'. EnigmaGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Context: 
MATRIX (1999)
12
The Oracle said it better than anyone in the movie. She did not tell Neo that he was the one like
everyone around him did. She told him that he would have to choose between him or Morpheus,
as a test to see if he would blindly believe her or challenge his fate by defying her.
She always knew he was the one, as she told Morpheus that he was the one who would find the
One, and that Trinity would be in love with the man who was the One. But until Neo knew it
himself, he was not the One. 

In [22]:
import gradio as gr


In [23]:

pages = pdf_loader.load_data("/content/BotGenesis/MATRIX_CRITICAL_REVIEW_cinema&philosophy_SE22UCSE106.pdf")
index = VectorStoreIndex.from_documents(pages)
retriever = VectorIndexRetriever(index=index, similarity_top_k=1)


In [24]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11

In [25]:
# Define Chatbot Response Function
# ========================
def chatbot_response(query):
    """Retrieves context and generates an AI response."""
    response = retriever.retrieve(query)
    context = "\n".join([node.text for node in response])

    # Create prompt with retrieved context
    prompt = f"""[INST]EnigmaGPT, your AI study buddy, will answer based on the provided context:
{context}
Question: {query} [/INST]"""

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [26]:
demo = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask me a question..."),
    outputs=gr.Textbox(),
    title="ColdBot",
    description="A Retrieval-Augmented Chatbot to answer questions based on given data."
)


In [27]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://76c7e3004111a9c4e7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
